# Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
from datetime import datetime
import os

# Seed for reproducing

In [ ]:
np.random.seed(888)
tf.random.set_seed(404)

# Constants

In [ ]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS

# Get the Data

In [ ]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)


# Data Preprocessing

In [ ]:
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_test = np.eye(NR_CLASSES)[y_test]

# Create validation dataset from training data

In [ ]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Neural Network Architecture

In [ ]:
nr_epochs = 50
learning_rate = 1e-3
n_hidden1 = 512
n_hidden2 = 64

# Define the Model

In [ ]:
model = Sequential([
    Flatten(input_shape=(TOTAL_INPUTS,)),
    Dense(n_hidden1, activation='relu'),
    Dropout(0.8),
    Dense(n_hidden2, activation='relu'),
    Dense(NR_CLASSES, activation='softmax')
])

# Compile the Model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# TensorBoard Setup

In [ ]:
log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Training the Model with TensorBoard

In [ ]:
history = model.fit(
    x_train, y_train, 
    epochs=nr_epochs, 
    batch_size=1000, 
    validation_data=(x_val, y_val), 
    callbacks=[tensorboard_callback]
)

# Saving the Model

In [ ]:
model.save('SavedModel/my_model')

# Making a Prediction

In [ ]:
def make_prediction(image_path):
    img = Image.open(image_path)
    bw = img.convert('L')
    img_array = np.invert(bw)
    test_img = img_array.ravel() / 255.0
    prediction = model.predict(np.array([test_img]))
    return np.argmax(prediction)

# Example of making a prediction

In [ ]:
prediction = make_prediction('MNIST/test_img.png')
print(f'Prediction for test image is {prediction}')

# Testing and Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy:.2%}")